In [2]:
from __future__ import print_function

In [3]:
import sys
!{sys.executable} -m pip install textblob

You should consider upgrading via the 'C:\Users\illa\anaconda3\python.exe -m pip install --upgrade pip' command.


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from io import StringIO

import nltk
from nltk.corpus import stopwords

import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.utils import shuffle
from sklearn.base import BaseEstimator, TransformerMixin

from lime import lime_text
from lime.lime_text import LimeTextExplainer

import re
import itertools  
%matplotlib inline
import warnings
warnings.simplefilter('ignore')

In [28]:
df_01 = pd.read_csv('df_settore_vantaggi_inconvenienti.csv')

In [7]:
df_01['vantaggi/inconvenienti'].isna().sum()

0

In [29]:
text_0 = df_01['vantaggi/inconvenienti']
y = df_01['bin_y'].tolist()

In [30]:
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

#regex rimuoviamo la punteggiatura
text = text_0.str.lower().str.replace('[^\w\s]','')
#seperiamo il testo con split
text = text.str.split()
# text = text.apply(lambda x: [lemmatizer.lemmatize(word) for sentence in x for word in sentence])
text = text.apply(lambda x: [lemmatizer.lemmatize(sent) for sent in x])
print(text[5])
print(text[0])

['hangout', 'meet', 'is', 'already', 'available', 'from', 'within', 'gmail', 'it', 'simple', 'to', 'add', 'to', 'a', 'meeting', 'invitation', 'from', 'gcal', 'or', 'to', 'launch', 'for', 'impromptu', 'meeting', 'from', 'the', 'gsuite', 'launchpad']
['ive', 'tried', 'other', 'solution', 'and', 'for', 'me', 'the', 'best', 'from', 'this', 'solution', 'is', 'how', 'easy', 'is', 'to', 'access', 'to', 'the', 'conference', 'for', 'my', 'customer', 'just', 'one', 'click', 'and', 'they', 'are', 'inside', 'the', 'conference']


In [31]:
from gensim.models.phrases import Phrases
from nltk.corpus import stopwords
stop_words = pd.read_csv('stop_list.txt', sep=" ", header=None)
stop.extend(['good', 'many', 'love', 'excellent', 'would'])

#min_count=entrambe le parole devono comparire almeno 5volte
#thresold=frequenza quante volte compaiono una dopo l'altra
#common_terms=vengono inseriti in mezzo se presenti, mai all'inizio o fine
bigram = Phrases(text, min_count=5, threshold=0.3, common_terms=stop)
print(bigram[text[5]])

['hangout', 'meet', 'is', 'already', 'available', 'from', 'within', 'gmail', 'it', 'simple', 'to', 'add', 'to', 'a', 'meeting', 'invitation', 'from', 'gcal', 'or', 'to', 'launch', 'for', 'impromptu', 'meeting', 'from', 'the', 'gsuite', 'launchpad']


In [32]:
print(bigram[text[0]])

['ive', 'tried', 'other', 'solution', 'and', 'for', 'me', 'the', 'best', 'from', 'this', 'solution', 'is', 'how', 'easy', 'is', 'to', 'access', 'to', 'the', 'conference', 'for', 'my', 'customer', 'just', 'one', 'click', 'and', 'they', 'are', 'inside', 'the', 'conference']


In [33]:
bigrams = [bigram[item] for item in text]
ngrams = [bigram[item] for item in bigrams] #4gram bigram del bigram
print(ngrams[0])

['ive', 'tried', 'other', 'solution', 'and', 'for', 'me', 'the', 'best', 'from', 'this', 'solution', 'is', 'how', 'easy', 'is', 'to', 'access', 'to', 'the', 'conference', 'for', 'my', 'customer', 'just', 'one', 'click', 'and', 'they', 'are', 'inside', 'the', 'conference']


In [34]:
from nltk.corpus import stopwords
stop_words = pd.read_csv('stop_list.txt', sep=" ", header=None)
stop = stop_words[0].tolist()
stop.extend(['good', 'bad', 'dont', 'many', 'love', 'excellent', 'would', 'perfect', 'even', 'great'])
print(ngrams[0])
train_sentences = []
for row in ngrams:
    train_sentences.append(' '.join([item for item in row if item not in stop]))
# train_sentences = [' '.join(item) for item in ngrams]
train_sentences[0]

['ive', 'tried', 'other', 'solution', 'and', 'for', 'me', 'the', 'best', 'from', 'this', 'solution', 'is', 'how', 'easy', 'is', 'to', 'access', 'to', 'the', 'conference', 'for', 'my', 'customer', 'just', 'one', 'click', 'and', 'they', 'are', 'inside', 'the', 'conference']


'tried solution solution access conference customer click inside conference'

In [35]:
df_01['vantaggi/svantaggi'] = train_sentences
df_01.to_csv('df_settore_vantaggi_svantaggi_binary.csv', index=None)

In [38]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#limitare le features per overfitting
#vectorizer = CountVectorizer(ngram_range=(1, 1), max_features=1000)
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=1000)
X = vectorizer.fit_transform(train_sentences)
#riottengo il nome delle features che prima sono 
#state converite in chiavi numeriche
feature_names = vectorizer.get_feature_names()
#X = X.toarray()
#le porti in nparray
X = np.array(X)
y = np.array(y)

In [39]:
from sklearn.model_selection import train_test_split
#il dataset è sbilanciato, riporto con stratify la stessa proporzione 
#sia nel train che test
x_train, x_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

TypeError: Singleton array array(<39436x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 265458 stored elements in Compressed Sparse Row format>,
      dtype=object) cannot be considered a valid collection.

In [10]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
stop_words = pd.read_csv('stop_list.txt', sep=" ", header=None)
stop = stop_words[0].tolist()
stop.extend(['good', 'bad', 'dont'])
lemmatizer = WordNetLemmatizer()

x = x_0.str.lower().str.replace('[^\w\s]','')
x = x.str.split()
x = x.apply(lambda x: [lemmatizer.lemmatize(item) for item in x])
x = x.apply(lambda x: [item for item in x if item not in stop])

In [16]:
bigrams = [bigram[item] for item in text]
ngrams = [bigram[item] for item in bigrams] #4gram bigram del bigram
print(ngrams[0])

['ive_tried', 'other', 'solution', 'and', 'for', 'me', 'the', 'best', 'from', 'this', 'solution', 'is', 'how', 'easy', 'is', 'to', 'access', 'to', 'the', 'conference', 'for', 'my', 'customer', 'just', 'one_click', 'and', 'they', 'are', 'inside', 'the', 'conference']


In [19]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
#impostiamo lunghezza mx review
max_length = 20
#ogni parola del dizionario viene identificata da un numero
t = Tokenizer()
t.fit_on_texts(x)
#size + 1 perché len conta a partire da 0
vocab_size = len(t.word_index) + 1
#conto i vocaboli risultanti dal modello di embedding 
emb_size=embeddings.vector_size
#trasformo parole in numeri
encoded_docs = t.texts_to_sequences(x)
#se la frase è maggiore di 20parole la tronco, se inferiore aggiungo 0 fino a 20
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

x_train, x_test, y_train, y_test = train_test_split(padded_docs, y, test_size=0.3, random_state=42, stratify=y)

In [20]:
import numpy as np
import tensorflow as tf
tf.random.set_seed(1)

# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0, in questo caso 20 zeri
    weight_matrix = np.zeros((vocab_size, emb_size))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        if word in embeddings.vocab:
            weight_matrix[i] = embedding[word]
    return weight_matrix
#risostituisco a ciascun valore del token una lista di 20 embedding

In [ ]:
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, LSTM, Bidirectional, GRU, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# get vectors in the right order
embedding_vectors = get_weight_matrix(embeddings, t.word_index)
print(embedding_vectors.shape)
e = Embedding(vocab_size, emb_size, weights=[embedding_vectors], input_length=max_length, trainable=False)

#params
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
n_labels = 2

# create the model
def baseline_model(optimizer=opt):
    model = Sequential()
    model.add(e)
    model.add(LSTM(50, activation='relu', return_sequences=True))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(n_labels, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    print(model.summary())
    return model

model = KerasClassifier(build_fn=baseline_model)
batch_size = [32, 64, 128] #quanta porzione prendo del corpus
epochs = [5, 10, 20]
optimizer = ['Adadelta', 'Nadam']
#batch_size = [64]
#epochs = [5]
#optimizer = ['Adam']
param_grid = dict(batch_size=batch_size, epochs=epochs, optimizer=optimizer)

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='f1_weighted', verbose=2)
print('***CROSS VALIDATED GRID SEARCH***')
%time grid_result = grid.fit(x_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

(13985, 100)
***CROSS VALIDATED GRID SEARCH***
Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


In [27]:
from sklearn.metrics import classification_report, f1_score, accuracy_score
#results on the test set for the best model
best_model = grid.best_estimator_
y_pred = best_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print('***RESULTS ON TEST SET***')
print("accuracy_score", accuracy)
print("f1_score", f1)
print('\n')
print(classification_report(y_test, y_pred))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
***RESULTS ON TEST SET***
accuracy_score 0.884600465477114
f1_score 0.9385838150289019


              precision    recall  f1-score   support

           0       0.26      0.03      0.05      1135
           1       0.89      0.99      0.94      9177

    accuracy                           0.88     10312
   macro avg       0.57      0.51      0.49     10312
weighted avg       0.82      0.88      0.84     10312

